In [2]:
import warnings

# Imports
import pandas as pd
import numpy as np
from datasets import load_dataset
import nltk

In [3]:
dataset = load_dataset("stanfordnlp/imdb")

In [4]:
df_train = pd.DataFrame(dataset['train'])
df_train

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0
...,...,...
24995,A hit at the time but now better categorised a...,1
24996,I love this movie like no other. Another time ...,1
24997,This film and it's sequel Barry Mckenzie holds...,1
24998,'The Adventures Of Barry McKenzie' started lif...,1


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
sentences = [text for text in df_train['text']]
countVectorizer = CountVectorizer()
X_test = countVectorizer.fit_transform(sentences)
X_test

<25000x74849 sparse matrix of type '<class 'numpy.int64'>'
	with 3445861 stored elements in Compressed Sparse Row format>

In [6]:
Y_test = np.array(df_train['label'])
Y_test

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [7]:
# MLP classifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
warnings.filterwarnings('ignore')
seed = 1234

In [ ]:
params = {
    "hidden_layer_sizes": np.arange(10, 250, 10),
    "activation": ['identity', 'logistic', 'tanh', 'relu'],
    "solver": ['lbfgs', 'sgd', 'adam'],
    "alpha": np.arange(0.01, 0.2, 0.02),
    "batch_size": np.arange(50, 201, 25),
    "learning_rate": ['constant', 'invscaling', 'adaptive'],
    "learning_rate_init": np.arange(0.001, 0.011, 0.001),
    "max_iter": np.arange(100, 301, 25),
    "random_state": [seed]
}
mlp = MLPClassifier()
clf = GridSearchCV(mlp, param_grid = params)
print("Running...")
clf.fit(X_test[:10], Y_test[:10])
print("Finished")

Running...


C:\Users\jerem\UniversityClassProjects\CS178\venv\hw1-anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(
C:\Users\jerem\UniversityClassProjects\CS178\venv\hw1-anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(
C:\Users\jerem\UniversityClassProjects\CS178\venv\hw1-anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(
C:\Users\jerem\UniversityClassProjects\CS178\venv\hw1-anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(
C:\Users